+ API 출처: [공공데이터포털 기상청_단기예보 조회서비스](https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15084084)

In [1]:
import requests
from datetime import datetime, timedelta

In [2]:
# 현재 시간 기준으로 base_date, base_time 반환하는 함수
def get_base_date():
    now = datetime.now() - timedelta(hours=3)   # 더 많은 데이터를 가져오기 위해 현재시간에서 3시간을 뺐음
    # 현재 시간이 02:10 이전이라면
    if now.hour < 2 or (now.hour == 2 and now.minute <= 10):
        base_date = (now - timedelta(days=1)).strftime('%Y%m%d')
        base_time = "2300"
    else:
        base_date = now.strftime('%Y%m%d')
        # 02:11 ~ 05:10
        if now.hour < 5 or (now.hour == 5 and now.minute < 10):
            base_time = "0200"
        # 05:11 ~ 08:10
        elif now.hour < 8 or (now.hour == 8 and now.minute < 10):
            base_time = "0500"
        # 08:11 ~ 11:10
        elif now.hour < 11 or (now.hour == 11 and now.minute < 10):
            base_time = "0800"
        # 11:11 ~ 14:10
        elif now.hour < 14 or (now.hour == 14 and now.minute < 10):
            base_time = "1100"
        # 14:11 ~ 17:10
        elif now.hour < 17 or (now.hour == 17 and now.minute < 10):
            base_time = "1400"
        # 17:11 ~ 20:10
        elif now.hour < 20 or (now.hour == 20 and now.minute < 10):
            base_time = "1700"
        # 20:11 ~ 23:10
        elif now.hour < 23 or (now.hour == 23 and now.minute < 10):
            base_time = "2000"
        # 23:11 ~ 23:59
        else:
            base_time = "2300"
    
    return base_date, base_time

In [74]:
base_date, base_time = get_base_date()
print(base_date, ",", base_time)

# 격자 X, 격자 Y의 값 임시 전달, 서울특별시 강남구 수서동
nx = '62'
ny = '125'

20220120 , 1700


In [75]:
# 단기예보조회 불러오기
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
service_key = ""
params ={'serviceKey' : service_key, 'pageNo' : '1', 'numOfRows' : '500', 'dataType' : 'JSON', 'base_date' : base_date, 'base_time' : base_time, 'nx' : nx, 'ny' : ny }

response = requests.get(url=url, params=params)
#print(response.content)

In [76]:
import json
#from IPython.display import display
from pprint import pprint

In [77]:
data = response.json()

In [78]:
data = response.json()

#display(items)
#pprint(data)

data = data['response']['body']['items']['item']
#pprint(data)

In [79]:
categorys = {
    "POP": "강수확률",
    "PTY": "강수형태",
    "PCP": "1시간 강수량",
    "REH": "습도",
    "SNO": "1시간 신적설",
    "SKY": "하늘상태",
    "TMP": "1시간 기온",
    "TMN": "일 최저기온",
    "TMX": "일 최고기온",
    "UUU": "풍속(동서성분)",
    "VVV": "풍속(남북성분)",
    "WAV": "파고",
    "VEC": "풍향",
    "WSD": "풍속"
}

In [137]:
class hourFcst:
    def __init__(self, date, time, fcs):
        self.fcstDate = date
        self.fcstTime = time
        self.fcst = fcs

    def printFcst(self):
        print("날짜:", self.fcstDate)
        print("시간:", self.fcstTime)
        print("관측 내용")
        #pprint(self.fcst)

In [138]:
# 현재시간 + 1시간 해주는 함수 (fcstTime 용)
def plusHour(hour):
    hour = int(hour) + 100

    if hour < 1000:
        hour = "0" + str(hour)

    if hour == 2400:
        hour = "0000"

    return str(hour)

In [194]:
# 현재시간 -1시간 해주는 함수
def minusHour(hour):
    hour = int(hour) - 100

    if hour < 1000:
        if hour == 0:
            hour = "0000"
        elif hour > 0:
            hour = "0" + str(hour)
        else:
            hour = "2300"

    return hour

In [196]:
fcsts = []
fcst = {}
nowTime = plusHour(base_time)
for item in data:
    if nowTime == item['fcstTime']:
        fcst[categorys[item['category']]] = item['fcstValue']
    else:
        #print("날짜:", item["fcstDate"], "/ 시간:", item["fcstTime"])
        fcsts.append(hourFcst(item['fcstDate'], minusHour(item['fcstTime']), fcst))
        nowTime = plusHour(nowTime)
        fcst = {}

In [198]:
for f in fcsts:
    f.printFcst()
    print("=========================")

날짜: 20220120
시간: 1800
관측 내용
날짜: 20220120
시간: 1900
관측 내용
날짜: 20220120
시간: 2000
관측 내용
날짜: 20220120
시간: 2100
관측 내용
날짜: 20220120
시간: 2200
관측 내용
날짜: 20220121
시간: 2300
관측 내용
날짜: 20220121
시간: 0000
관측 내용
날짜: 20220121
시간: 0100
관측 내용
날짜: 20220121
시간: 0200
관측 내용
날짜: 20220121
시간: 0300
관측 내용
날짜: 20220121
시간: 0400
관측 내용
날짜: 20220121
시간: 0500
관측 내용
날짜: 20220121
시간: 0600
관측 내용
날짜: 20220121
시간: 0700
관측 내용
날짜: 20220121
시간: 0800
관측 내용
날짜: 20220121
시간: 0900
관측 내용
날짜: 20220121
시간: 1000
관측 내용
날짜: 20220121
시간: 1100
관측 내용
날짜: 20220121
시간: 1200
관측 내용
날짜: 20220121
시간: 1300
관측 내용
날짜: 20220121
시간: 1400
관측 내용
날짜: 20220121
시간: 1500
관측 내용
날짜: 20220121
시간: 1600
관측 내용
날짜: 20220121
시간: 1700
관측 내용
날짜: 20220121
시간: 1800
관측 내용
날짜: 20220121
시간: 1900
관측 내용
날짜: 20220121
시간: 2000
관측 내용
날짜: 20220121
시간: 2100
관측 내용
날짜: 20220121
시간: 2200
관측 내용
날짜: 20220122
시간: 2300
관측 내용
날짜: 20220122
시간: 0000
관측 내용
날짜: 20220122
시간: 0100
관측 내용
날짜: 20220122
시간: 0200
관측 내용
날짜: 20220122
시간: 0300
관측 내용
날짜: 20220122
시간: 0400
관측 내용
날짜: 20220122
시간: 050